<a href="https://colab.research.google.com/github/elsa9421/Interactive-IPython-Demos/blob/main/PCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; 
from sklearn.decomposition import PCA
from math import atan,sin,cos,pi,sqrt
from ipywidgets import interact,interactive,interactive_output, fixed, interact_manual
import ipywidgets as widgets
import scipy.stats as stats



#Generate 2D data points

rng = np.random.RandomState(42)
X = np.dot(rng.rand(2, 2), rng.randn(2, 100)).T



pca = PCA(n_components=1).fit(X)
d_mag=np.linalg.norm(pca.components_)
d_angle=atan(pca.components_[0][1]/pca.components_[0][0])
d_angle=d_angle*(180/pi)


# ax2=plt.subplot(1,2,2)


def plot_portfolio(angle,X=X,mag=d_mag):
  '''
  The functionplots the 1D projection given 2D data.
  The angle of the projection line can be varied.

  Input:
  -X : input 2D data of shape (N,2) 


  '''

  # Plot 2D data points
  sns.set()
 
  plt.figure(figsize=(14,7))

  plt.subplot(1,2,1)
  plt.scatter(X[:, 0], X[:, 1], alpha=0.3,label='2-D Data')
  plt.axis('equal')

  angle=angle*(pi/180)
  
  eig_vec1=np.array([round(mag*(cos(angle)),5),round(mag*(sin(angle)),5)])

   
  # To plot the projection line  -> pca.components_ are principal components of X
  xmin,xmax=plt.xlim()
  ymin,ymax=plt.ylim()
  plt.xlim(xmin,xmax)
  plt.ylim(ymin,ymax)

  xvec=np.linspace(xmin,xmax)
  #origin = [0, 0]
  #plt.quiver(*origin, *eig_vec1, color=['r'],scale=3,alpha=1)
  if eig_vec1[0]==0:
     plt.axvline(x=0,alpha=0.8,color='k',label='Projection hyperplane')
     s=np.array([0,1])
     s_new=np.array([[0,1],]*len(X))
  else:
    slope=eig_vec1[1]/eig_vec1[0]
    plt.plot(xvec,slope*xvec,'k',alpha=0.8,label='Projection hyperplane')
    s=np.array([1,slope])  # of shape(2,) gives vector of projection line
    s_new=np.array([[1,slope],]*len(X)) # of shape(200,2)



    
    

   # Projection of datapoint on projection line=> (<x,s>/<s,s>).s
  s_norm=np.dot(s.T,s)     # <s,s>
  x_new=np.divide(np.dot(X,s),s_norm).reshape(-1,1)  # of shape(200,) 
  x_new1=np.concatenate((x_new,x_new),axis=1)
  out=np.multiply(s_new,x_new1)

  plt.scatter(out[:,0], out[:,1], alpha=0.4,color='red',label="Data projected on hyperplane")
  plt.legend()

  
  rot = np.array([[np.cos(angle), -np.sin(angle)],
                [np.sin(angle), np.cos(angle)]])
  rotated_data = np.matmul(out, rot)

  variance=np.var(rotated_data)
  print("Variance=",variance)
  sigma = sqrt(variance)
  mean=np.mean(rotated_data)



  #Subplot2 
  plt.subplot(1,2,2)
  ymin, ymax =-0.1, 1.8

  plt.xlim(xmin,xmax)
  plt.ylim(ymin, ymax)

  
  plt.scatter(rotated_data[:,0],rotated_data[:,1],alpha=0.4,color='red')
  plt.plot(xvec, stats.norm.pdf(xvec, mean, sigma))
  plt.xlabel("First Principal Component")
  plt.ylabel("PDF")

  







angle_slider=widgets.FloatSlider(value=d_angle,
    min=0,
    max=360,
    step=5,
    description='w angle (deg)',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True)

angle_text=widgets.FloatText(value=d_angle,
    min=0,
    max=360,
    step=5,
    description='w angle (deg)',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True)

widgets.link((angle_slider, 'value'), (angle_text, 'value'))
angle_widget=widgets.HBox([angle_slider,angle_text])



out=interactive_output(plot_portfolio,{"angle":angle_text})

%time display(angle_widget,out)
None


Output()

CPU times: user 4.95 ms, sys: 1.19 ms, total: 6.14 ms
Wall time: 6.35 ms
